# Scanning Session Ingestion

## Setup

### Connect to the database

If you are don't have your login information, contact the administrator.

Using local config file (see [01_pipeline](./01_pipeline.ipynb)):

In [1]:
import os
# change to the upper level folder to detect dj_local_conf.json
if os.path.basename(os.getcwd())=='notebooks': os.chdir('..')
assert os.path.basename(os.getcwd())=='adamacs', ("Please move to the main directory")
import datajoint as dj; dj.conn()

Please enter DataJoint username: root
Please enter DataJoint password: ········
Connecting root@localhost:3306


OperationalError: (2003, "Can't connect to MySQL server on 'localhost' ([WinError 10061] No connection could be made because the target machine actively refused it)")

Manual entry:

In [ ]:
# Manual Entry
import datajoint as dj; import getpass
dj.config['database.host'] = '172.26.128.53'        # Put the server name between these apostrophe
dj.config['database.user'] = 'danielmk'             # Put your user name between these apostrophe
dj.config['database.password'] = getpass.getpass()  # Put your password in the prompt
dj.conn()

### Imports and activation

Importing schema from `adamacs.pipeline` automatically activates relevant schema.

In [2]:
import datajoint as dj
from adamacs.pipeline import subject, session, surgery, scan, event, trial
from adamacs import utility
from adamacs.ingest import behavior as ibe

Assign easy names for relevant tables

In [3]:
sub, lab, protocol, line, mutation, user, project, subject_genotype, subject_death = (
    subject.Subject(), subject.Lab(), subject.Protocol(), subject.Line(), 
    subject.Mutation(), subject.User(), subject.Project(), subject.SubjectGenotype(), 
    subject.SubjectDeath())

In [4]:
event_types = ['main_track_gate', 'shutter', 'mini2p_frames', 'mini2p_lines', 'mini2p_volumes', 'aux_bpod_cam',
               'aux_bpod_visual', 'aux_bpod_reward', 'aux_bpod_tone']

for e in event_types:
    event.EventType.insert1({'event_type': e, 'event_type_description': ''}, skip_duplicates=True,)

## Ingesting behavior

In [5]:
ibe.ingest_aux("sess9FB2LN5C")

In [8]:
event.Event()

session_id,event_type,event_start_time (second) relative to recording start,event_end_time (second) relative to recording start
sess9FB2LN5C,aux_bpod_cam,0.0,0.005
sess9FB2LN5C,aux_bpod_cam,0.01665,0.02165
sess9FB2LN5C,aux_bpod_cam,0.03335,0.03835
sess9FB2LN5C,aux_bpod_cam,0.05,0.055
sess9FB2LN5C,aux_bpod_cam,0.06665,0.07165
sess9FB2LN5C,aux_bpod_cam,0.08335,0.08835
sess9FB2LN5C,aux_bpod_cam,0.1,0.105
sess9FB2LN5C,aux_bpod_cam,0.11665,0.12165
sess9FB2LN5C,aux_bpod_cam,0.13335,0.13835
sess9FB2LN5C,aux_bpod_cam,0.15,0.155


In [5]:
isess.ingest_session_scan('sess9FB2LN5C', verbose=True)

Number of scans found: 1
[['scan9FB2LN5C']]
['DB_WEZ-8701_2022-03-18_scan9FB2LN5C_sess9FB2LN5C']


C:\Users\Daniel\AppData\Local\Temp\ipykernel_14768\2690977035.py:1: UserWarning: 
Skipped existing session row: ('sess9FB2LN5C', <module 'adamacs.schemas.subject' from 'C:\\Users\\Daniel\\repos\\adamacs\\adamacs\\schemas\\subject.py'>, '2022-03-18')
  isess.ingest_session_scan('sess9FB2LN5C', verbose=True)
C:\Users\Daniel\AppData\Local\Temp\ipykernel_14768\2690977035.py:1: UserWarning: 
Skipped existing SessionDirectory: ('sess9FB2LN5C', WindowsPath('C:/datajoint/DB_WEZ-8701_2022-03-18_scan9FB2LN5C_sess9FB2LN5C'))
  isess.ingest_session_scan('sess9FB2LN5C', verbose=True)
C:\Users\Daniel\AppData\Local\Temp\ipykernel_14768\2690977035.py:1: UserWarning: 
Skipped existing SessionUser row: ('sess9FB2LN5C', 1)
  isess.ingest_session_scan('sess9FB2LN5C', verbose=True)
C:\Users\Daniel\AppData\Local\Temp\ipykernel_14768\2690977035.py:1: UserWarning: 
Skipped existing scan: scan9FB2LN5C
  isess.ingest_session_scan('sess9FB2LN5C', verbose=True)
C:\Users\Daniel\AppData\Local\Temp\ipykernel_14768\2

In [7]:
session.Session * session.SessionDirectory

session_id,subject PyRat import uses this for earmark value,session_datetime,session_dir Path to the data directory for a session,user_id
sess9FB2LN5C,WEZ-8701,2022-03-18 00:00:00,C:\datajoint\DB_WEZ-8701_2022-03-18_scan9FB2LN5C_sess9FB2LN5C,1


In [8]:
key='scan9FB2LN5C'
(scan.Scan & f'scan_id=\"{key}\"')

session_id,scan_id,scanner,acq_software,scan_notes free-notes
sess9FB2LN5C,scan9FB2LN5C,Equipment,ScanImage,


In [5]:
scan.ScanInfo.populate()

<class 'list'> ['C:/datajoint/DB_WEZ-8701_2022-03-18_scan9FB2LN5C_sess9FB2LN5C/scan9FB2LN5C_DB_WEZ-8701_02027_00001.tif']


`CB DEV NOTE:` In demo file, there are no `scan.motor_position_at_zero` values. Set to 0 here to avoid downstream type errors when adding. Future would should set these to null when not present and revise downstream code.

In [6]:
scan.ScanInfo()

session_id,scan_id,nfields number of fields,nchannels number of channels,ndepths Number of scanning depths (planes),nframes number of recorded frames,nrois number of ROIs (see scanimage's multi ROI imaging),x (um) ScanImage's 0 point in the motor coordinate system,y (um) ScanImage's 0 point in the motor coordinate system,z (um) ScanImage's 0 point in the motor coordinate system,fps (Hz) frames per second - Volumetric Scan Rate,bidirectional true = bidirectional scanning,usecs_per_line microseconds per scan line,fill_fraction raster scan temporal fill fraction (see scanimage),scan_datetime datetime of the scan,scan_duration (seconds) duration of the scan
sess9FB2LN5C,scan9FB2LN5C,1,2,1,5000,0,nan,nan,nan,15.3846,1,250.0,0.712867,None,325.0


In [7]:
scan.ScanInfo.Field()

session_id,scan_id,field_idx,px_height height in pixels,px_width width in pixels,um_height height in microns,um_width width in microns,field_x (um) center of field in the motor coordinate system,field_y (um) center of field in the motor coordinate system,field_z (um) relative depth of field,delay_image (ms) delay between the start of the scan and pixels in this field,roi the scanning roi (as recorded in the acquisition software) containing this field - only relevant to mesoscale scans
sess9FB2LN5C,scan9FB2LN5C,0,256,256,202.5,202.5,nan,nan,nan,=BLOB=,None


Note the relative path below:

In [8]:
scan.ScanInfo.ScanFile()

session_id,scan_id,file_path filepath relative to root data directory
sess9FB2LN5C,scan9FB2LN5C,DB_WEZ-8701_2022-03-18_scan9FB2LN5C_sess9FB2LN5C
